## Dialogue summarization task using prompt engineering

In [1]:
!pip install torch
!pip install torchdata
!pip install transformers[sentencepiece]
!pip install sentencepiece

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
from datasets import load_dataset
from transformers import GenerationConfig
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration

In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

In [5]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

#### I - Summarize Dialogue without Prompt Engineering

In [7]:
dialogue = dataset['test']['dialogue'][0]
summary = dataset['test']['summary'][0]

inputs = tokenizer(dialogue, return_tensors='pt').input_ids

generation_config = GenerationConfig(max_new_tokens=50)

response = model.generate(
        inputs, 
        generation_config=generation_config
)[0]

output = tokenizer.decode(
    response, 
    skip_special_tokens=True
)

print(f'INPUT PROMPT:\n{dialogue}')
print('------------------------------------------------')
print(f'BASELINE HUMAN SUMMARY:\n{output}')

INPUT PROMPT:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue wit

#### II - Summarize Dialogue with an Instruction Prompt

**1 - Zero Shot Inference with an Instruction Prompt**

In [8]:
dialogue = dataset['test']['dialogue'][0]
summary = dataset['test']['summary'][0]

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt').input_ids

generation_config = GenerationConfig(max_new_tokens=50)

response = model.generate(
        inputs, 
        generation_config=generation_config
)[0]

output = tokenizer.decode(
    response, 
    skip_special_tokens=True
)

print(f'INPUT PROMPT:\n{dialogue}')
print('------------------------------------------------')
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

INPUT PROMPT:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue wit

**2 - One Shot Inference with an Instruction Prompt**

In [9]:
dialogue = dataset['test']['dialogue'][0]
summary = dataset['test']['summary'][0]
dialogue_exp = dataset['test']['dialogue'][2]
summary_exp = dataset['test']['summary'][2]

prompt = ''

prompt += f"""
Dialogue:

{dialogue_exp}

What was going on?
{summary_exp}
"""

prompt += f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt').input_ids

generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)

response = model.generate(
        inputs, 
        generation_config=generation_config
)[0]

output = tokenizer.decode(
    response, 
    skip_special_tokens=True
)

print(f'INPUT PROMPT:\n{dialogue}')
print('------------------------------------------------')
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (764 > 512). Running this sequence through the model will result in indexing errors


INPUT PROMPT:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue wit

**3 - Few Shot Inference with an Instruction Prompt**

In [10]:
dialogue = dataset['test']['dialogue'][0]
summary = dataset['test']['summary'][0]

prompt = ''

for i in range(1,4):
    
    dialogue_exp = dataset['test']['dialogue'][i]
    summary_exp = dataset['test']['summary'][i]
    prompt += f"""
        Dialogue:
        
        {dialogue_exp}
        
        What was going on?
        {summary_exp}
    """

prompt += f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt').input_ids

generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.3)

response = model.generate(
        inputs, 
        generation_config=generation_config
)[0]

output = tokenizer.decode(
    response, 
    skip_special_tokens=True
)

print(f'INPUT PROMPT:\n{dialogue}')
print('------------------------------------------------')
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

INPUT PROMPT:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue wit

In this case, few shot did not provide much of an improvement over one shot inference. And, anything above 5 or 6 shot will typically not help much, either. Also, you need to make sure that you do not exceed the model's input-context length which, in our case, if 512 tokens. Anything above the context length will be ignored.